<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Effects Model with Genomic Data</font> 
</div>

In [1]:
include("../src/JWAS.jl")

JWAS

In [2]:
using DataFrames,CSV,JWAS
using JWAS: Datasets,misc

<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [3]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [6]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [7]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multiple Traits Analyses with Marker Information</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [8]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [9]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [10]:
model1 = build_model(model_equations,R);

In [11]:
set_covariate(model1,"age");

In [12]:
add_genotypes(model1,genofile,G,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [13]:
Pi=Dict([1.0; 1.0]=>0.7,[1.0;0.0]=>0.1,[0.0,1.0]=>0.1,[0.0; 0.0]=>0.1)
out = runMCMC(model1,data,Pi=Pi,chain_length=5000,methods="BayesC",
estimatePi=true,output_samples_frequency=5);

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[10.9589 0.626223; 0.626223 1.09589].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          5
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.1,[1.0, 0.0]=>0.1,[1.0, 1.0]=>0.7,[0.0, 0.0]=>0.1)
estimatePi                                     true

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.0

INFO: The file MCMC_samples_for_marker_effects_CW (_variance) was created to save MCMC samples for marker effects (variances).
running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:03


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [14]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 7 entries. Keys:
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "MCMC samples for residual covariance matrix"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [15]:
file1="MCMC_samples_for_marker_effects_BW.txt"
file2="MCMC_samples_for_marker_effects_CW.txt";

In [16]:
get_breeding_values(model1,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV      │ PEV     │
├─────┼────┼──────────┼─────────┤
│ 1   │ S1 │ -2.13284 │ 12.8013 │
│ 2   │ D1 │ 6.06635  │ 66.0589 │
│ 3   │ O1 │ 5.16684  │ 91.0337 │
│ 4   │ O3 │ -9.10035 │ 63.7778 │            
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV       │ PEV      │
├─────┼────┼───────────┼──────────┤
│ 1   │ S1 │ -0.350336 │ 0.573071 │
│ 2   │ D1 │ 1.34708   │ 4.10615  │
│ 3   │ O1 │ 1.02729   │ 4.40039  │
│ 4   │ O3 │ -2.02403  │ 4.36956  │

In [17]:
samples4G=get_additive_genetic_variances(model1,file1,file2);

samples4R=out["MCMC samples for residual covariance matrix"];

samples4h2=get_heritability(reformat(samples4G),reformat(samples4R));

In [18]:
writedlm("out3.txt",samples4G)

In [19]:
samples_genetic_correlation=get_correlations(reformat(samples4G));

In [20]:
writedlm("out.G.txt",reformat(reformat(samples4G)))

In [21]:
out=readdlm("out.G.txt")

4×1000 Array{Float64,2}:
 232.679    440.241    170.74      …   7.62933    2.33657   0.251824
  36.5477    39.7075     2.01809      -0.765663  -1.56052  -0.547344
  36.5477    39.7075     2.01809      -0.765663  -1.56052  -0.547344
   7.56702    4.28524    0.893903      2.69818    1.66174   3.64055 

In [22]:
reformat(out)

1000-element Array{Array{Float64,2},1}:
 [232.679 36.5477; 36.5477 7.56702]     
 [440.241 39.7075; 39.7075 4.28524]     
 [170.74 2.01809; 2.01809 0.893903]     
 [25.0142 -9.55532; -9.55532 4.12834]   
 [60.9211 -25.5503; -25.5503 12.2435]   
 [2.39062 4.88461; 4.88461 29.9666]     
 [23.899 -0.492875; -0.492875 21.6686]  
 [0.0 0.0; 0.0 5.70531]                 
 [1.1561 0.468535; 0.468535 0.900379]   
 [15.2775 2.40606; 2.40606 0.438716]    
 [1.01231 -0.402693; -0.402693 0.58772] 
 [3.86834 -1.20909; -1.20909 0.682991]  
 [20.7986 -3.39071; -3.39071 0.741051]  
 ⋮                                      
 [102.996 12.6577; 12.6577 1.62105]     
 [118.022 25.12; 25.12 5.41303]         
 [19.7451 2.73923; 2.73923 0.902371]    
 [12.4853 2.9555; 2.9555 2.04112]       
 [18.1989 6.15715; 6.15715 4.35779]     
 [5.0138 -2.49922; -2.49922 1.31482]    
 [6.50866 -1.72672; -1.72672 0.873854]  
 [1.10598 -1.07584; -1.07584 1.4483]    
 [9.15866 0.0; 0.0 0.0]                 
 [7.62933 -0.7656

In [23]:
out10=reformat(samples4h2)

2×1000 Array{Float64,2}:
 0.41228   0.365318  0.233739   …  0.0810436  0.0485538  0.00284844
 0.311643  0.103666  0.0240935     0.329418   0.3005     0.307536  

In [24]:
reformat(out,2)

1000-element Array{Array{Float64,2},1}:
 [232.679 36.5477; 36.5477 7.56702]     
 [440.241 39.7075; 39.7075 4.28524]     
 [170.74 2.01809; 2.01809 0.893903]     
 [25.0142 -9.55532; -9.55532 4.12834]   
 [60.9211 -25.5503; -25.5503 12.2435]   
 [2.39062 4.88461; 4.88461 29.9666]     
 [23.899 -0.492875; -0.492875 21.6686]  
 [0.0 0.0; 0.0 5.70531]                 
 [1.1561 0.468535; 0.468535 0.900379]   
 [15.2775 2.40606; 2.40606 0.438716]    
 [1.01231 -0.402693; -0.402693 0.58772] 
 [3.86834 -1.20909; -1.20909 0.682991]  
 [20.7986 -3.39071; -3.39071 0.741051]  
 ⋮                                      
 [102.996 12.6577; 12.6577 1.62105]     
 [118.022 25.12; 25.12 5.41303]         
 [19.7451 2.73923; 2.73923 0.902371]    
 [12.4853 2.9555; 2.9555 2.04112]       
 [18.1989 6.15715; 6.15715 4.35779]     
 [5.0138 -2.49922; -2.49922 1.31482]    
 [6.50866 -1.72672; -1.72672 0.873854]  
 [1.10598 -1.07584; -1.07584 1.4483]    
 [9.15866 0.0; 0.0 0.0]                 
 [7.62933 -0.7656

In [25]:
#genetic correlation between trait 1 and trait 2
report(reformat(samples4G),index=[1,2]);

Summary Stats:
Mean:           20.719119
Minimum:        -33.861859
1st Quartile:   0.000000
Median:         8.677151
3rd Quartile:   40.312995
Maximum:        143.288595
Length:         1000
Type:           Float64
nothing


<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. Multiple Traits Analyses with Marker Effects and Polygenic Effects</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [26]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";
model2          = build_model(model_equations,R);

set_covariate(model2,"age");

get pedigree information from a file

In [27]:
ped=get_pedigree(pedfile);

calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00

Finished!


In [28]:
GA = G*0.1
set_random(model2,"Animal",ped,GA)

In [29]:
GM = G*0.9
add_genotypes(model2,genofile,GM,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [30]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out2=runMCMC(model2,data,Pi=Pi,chain_length=5000,methods="BayesC");

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[15.7808 1.57808; 1.57808 1.57808].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.25,[1.0, 0.0]=>0.25,[1.0, 1.0]=>0.25,[0.0, 0.0]=>0.25)
estimatePi                                    false

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:02


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [31]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "Posterior mean of location parameters"

In [32]:
out2["Posterior mean of polygenic effects covariance matrix"]

2×2 Array{Float64,2}:
 5.75923   0.995464
 0.995464  0.422562

In [33]:
;ls

0.read_at_first.ipynb
1.1.Univariate_Linear_Mixed_Effects_Model.ipynb
1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb
2.1.Univariate_Linear_Additive_Genetic_Model.ipynb
2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb
3.1.Univariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb
3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb
4.miscellaneous_routines.ipynb
5.SSBR.ipynb
MCMC_samples_for_marker_effects_BW.txt
MCMC_samples_for_marker_effects_BW_variance.txt
MCMC_samples_for_marker_effects_CW.txt
MCMC_samples_for_marker_effects_CW_variance.txt
QTL.jpg
Theory
index.ipynb
out.G.txt
out3.txt
test.txt
